### Prerequisites

To run this .NET Interactive run book, you need a create an AppRegistration in Azure AD with the following settings
- Authentication -> Platform: Mobile and Desktop
- Authentication -> Platform -> RedirectUris: Msal...
- Authentication -> Supported Account Types: Single Tenant
- Authentication -> Advanced Settings -> Allow public client flows: True
- Press **Save**

- API Permissions -> Add a Permission -> Delegated Permisions -> MS Graph: `Application.ReadWrite.All`

We start by importing the necessary NuGet packages to be used throughout this runbook

In [ ]:
#r "nuget:Microsoft.Graph,4.25.0"
#r "nuget:Azure.Identity,1.6.0"

In [ ]:
using Microsoft.Graph;
using Azure.Identity;

var browserCreds = new InteractiveBrowserCredential( new InteractiveBrowserCredentialOptions
{
    TenantId = "b55f0c51-61a7-45c3-84df-33569b247796",
    ClientId = "7537790f-b619-4d30-a804-1c6b8b7f1523"
});

var scopes= new [] {"Application.ReadWrite.All", "User.Read" };
var graphClient = new GraphServiceClient(browserCreds, scopes);

### Create an API App Registration in Azure AD

Right now the scope is hardcoded to `api.read`. Feel free to change this to meet your needs

In [ ]:
var appName = "CM Demo API";
var apiScopeName = "api.read";
var scopeConsentMessage = "access the api as a reader";

var apiApp = GetApiApplication(appName, scopeConsentMessage, apiScopeName);

var apiAppRegistration = await graphClient.Applications
                            .Request()
                            .AddAsync(apiApp);

private async Task<string> GetTenantId()
{
    var organization = await graphClient.Organization
                                .Request()
                                .GetAsync();
    return organization.FirstOrDefault().Id;
}

Console.WriteLine($"Client Id: {apiAppRegistration.AppId}");
Console.WriteLine($"Domain: {apiAppRegistration.PublisherDomain}");
Console.WriteLine($"Tenant Id: {await GetTenantId()}");

private static Application GetApiApplication(
    string displayName, 
    string consentMessage,
    string scopeName) => new Application
{
    DisplayName = displayName,
    IdentifierUris = new List<String>()
    {
        "api://cm-demo"
    },
    Api = new ApiApplication
    {
        AcceptMappedClaims = null,
        KnownClientApplications = new List<Guid>()
        {
        },
        RequestedAccessTokenVersion = 2,
        Oauth2PermissionScopes = new List<PermissionScope>()
        {
            new PermissionScope
            {
                Id = Guid.NewGuid(),
                AdminConsentDescription = consentMessage,
                AdminConsentDisplayName = consentMessage,
                IsEnabled = true,
                Type = "User",
                UserConsentDescription = consentMessage,
                UserConsentDisplayName = consentMessage,
                Value = scopeName
            }
        },
        PreAuthorizedApplications = new List<PreAuthorizedApplication>()
        {
        }
    }
};

### Create the Service Principal
Next we need to create a Service Principal that maps to the App Registration. 
This registers the app with the current Tenant and make the App Registration available to 
the tenant's apps and users to authenticate against and/or acquire tokens.

In [ ]:
var servicePrincipal = await graphClient.ServicePrincipals
                            .Request()
                            .AddAsync(CreateServicePrincipal(apiAppRegistration.AppId));

private static ServicePrincipal CreateServicePrincipal(string appId) => new ServicePrincipal
{
    AppId = appId
};

### App Registration
Create a client App Registration for a server-side **web app** (.NET, Node, Java etc).

Users signing in to the client app, will be able to request an access token for the API.
You'll need to set the following two properties before running this script:
- `clientAppName` -> the name for your App Registration
- `redirectUri` -> the URI where your app expects the returned tokens. For OpenAPI the expected Redirect URI 
 is `https://localhost:5001/swagger/oauth2-redirect.html`

In [ ]:
using Microsoft.Graph;

var clientAppName = "CM SPA Demo";
var redirectUri = "https://localhost/index.js";

var clientApp = GetClientApplication(
    apiAppRegistration,
    clientAppName,
    redirectUri);

var clientAppRegistration = await graphClient.Applications
                                .Request()
                                .AddAsync(clientApp);

Console.WriteLine($"Client Id: {clientAppRegistration.AppId}");

private static Application GetClientApplication(
    Application app,
    string displayName, 
    string redirectUri) => new Application
{
    DisplayName = displayName,
    Spa = new SpaApplication
    {
        RedirectUris = new List<string>(){redirectUri}
    },
    RequiredResourceAccess = new List<RequiredResourceAccess>()
    {
        new RequiredResourceAccess
        {
            ResourceAppId = app.AppId,
            ResourceAccess = new List<ResourceAccess>()
            {
                new ResourceAccess
                {
                    Id = app.Api.Oauth2PermissionScopes.FirstOrDefault().Id,
                    Type = "Scope"
                }
            }
        },
        new RequiredResourceAccess
        {
            // OIDC - Graph scope (user.read)
            ResourceAppId = "00000003-0000-0000-c000-000000000000",
            ResourceAccess = new List<ResourceAccess>()
            {
                new ResourceAccess
                {     
                    Id = new Guid("e1fe6dd8ba314d6189e788639da4683d"),
                    Type = "Scope"
                }
            }
        }
    } 
};